In [1]:
import random
import functools
import pyrankvote
from array import *
import gurobipy as gp
from gurobipy import GRB
import itertools
from stvpoll.scottish_stv import ScottishSTV
import math
import pandas as pd
import math
import copy
import numpy
from decimal import *
from decimal import Decimal
from pyrankvote import Candidate, Ballot
import inspect
from stvpoll.__init__ import ElectionRound

In [2]:
n = 40
#number of votes
#length of each ballot
most=8
x=4
#number of the candidates
#m = rndomBallotGen(n,l,x)

In [3]:
def zetta(order,k,r):
    cand=order[0:r]
 #   print(cand)
    
    lst = list(order)
    perm=list(itertools.permutations(lst))
    perm.sort()
    ans=[]
    ek=order[k]
    for i in range(len(perm)):
        ind=perm[i].index(ek)
        ok=True
    #    print(i,perm[i],ind)
        for j in range(ind+1,len(order)):
            if((perm[i][j] not in cand)):
            #    print(perm[j])
                ok=False
                break
        if(ok):
            ans.append(i)
    
    
        
    return ans

In [4]:
print(zetta([2,0,3,1],3,1))

[3, 4, 5, 13, 16, 18, 19, 22]


In [5]:
def rndomBallotGen(most):
    votes  = []
    for i in range(math.factorial(x)):
        votes.append(random.randint(0,most))
        
    return votes

In [6]:
def ILP(votes,order):
    #define model
    largeValue = 1000
    noc=len(order)
    model = gp.Model("margin")
    model.setParam("OutputFlag",0)
    ted=math.factorial(noc)
#    model.params.NonConvex = 2
    b = model.addVar(vtype=GRB.INTEGER,name='b')
    # x[i,j] = a binary variable that is 1 iff candidate j is eliminated before candidate i
    #ai,j a variable representing the number of ballots < i, j > that are added
    p = model.addVars(ted,vtype=GRB.INTEGER,name='p')
    m = model.addVars(ted,vtype=GRB.INTEGER,name='m')
    y = model.addVars(ted,vtype=GRB.INTEGER,name='y')
    
    for i in range(ted):
        model.addConstr(y[i]<=sum(votes))
        model.addConstr(votes[i]+p[i]-m[i]==y[i])
        model.addConstr(0<=y[i])
        model.addConstr(m[i]<=votes[i])
        model.addConstr(0<=m[i])
        model.addConstr(0<=p[i])
    model.addConstr(sum(p[i] for i in range(ted)) == sum(m[i] for i in range(ted)))
    lst = copy.deepcopy(order)
    perm=list(itertools.permutations(lst))
    perm.sort()
    for i in range(noc):
        for j in range(i+1,noc):
            set1=zetta(order,i,i)
            set2=zetta(order,j,i)
            p1=gp.quicksum(y[l] for l in set1)
            p2=gp.quicksum(y[l] for l in set2)
       #     print(set1,set2)
            model.addConstr(p1<=p2)
    model.addConstr(b == gp.quicksum(p[i] for i in range(ted)))
    model.setObjective(b, GRB.MINIMIZE)
    model.optimize()
    
    pa=list(p[i].X for i in range(ted))
    ya=list(y[i].X for i in range(ted))
    ma=list(m[i].X for i in range(ted))
   
  
    #print(myvars)

    return model.getObjective().getValue()

In [7]:
# lst = list(range(x))
# perm=list(itertools.permutations(lst))
# perm.sort()
# print(perm)

In [8]:
def margin(votes,w):
    ans=1000*1000
    order=[]
    lst = list(range(x-1))
    perm=list(itertools.permutations(lst))
    perm.sort()
    for i in perm:
        i=list(i)
        for j in range(x-1):
            if(i[j]>=w):
                i[j]+=1
        i.append(w)
        jav=ILP(votes,i)
        if(ans>jav):
            ans=jav
            order=i
        
    return ans,order

In [9]:
def convert(a,b):
    ans=copy.deepcopy(a)
 #   print("a=",a)
    for i in a:
        if i not in b:
            ans.remove(i)
    return ans

In [10]:
votes=rndomBallotGen(most)
print(votes)
print(margin(votes,3))

[5, 4, 3, 7, 2, 1, 6, 0, 7, 6, 4, 8, 0, 5, 4, 5, 2, 4, 7, 6, 5, 2, 5, 6]
Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-26
(5.0, [0, 1, 2, 3])


In [11]:
def tolist(listuple):
    ans=[]
    for i in ((listuple)):
        ans.append(list(i))
    return ans

In [12]:
def optmar(votes,w):
    ans=1000*1000
    order=[]
    pr=[]
    lst = list(i for i in range(x))
#    print("lst=",lst)
    perm=list(itertools.permutations(lst))
    perm.sort()
    perm=copy.deepcopy(perm)
#    print("perm=",perm)
    perm=copy.deepcopy(tolist(perm))
#    print(perm[0])
    pr.append([0,[w]])
    while(len(pr)):
        it=copy.deepcopy(pr[0])
        if(len(it[1])==x):
#            print(pr)
            return it[0],it[1]
        for i in range(x):
            if(i not in it[1]):
                it2=[i,*it[1]]
                secprm=list(itertools.permutations(it2))
                secprm.sort()
                secprm=copy.deepcopy(tolist(secprm))
                vt=[0]*math.factorial(len(it2))
            #    print("it2 =",it2)
                for j in range(len(perm)):
                    aftcnv=copy.deepcopy(convert((perm[j]),it2))
           #         print(aftcnv)
           #         print(secprm)
                    vt[secprm.index(aftcnv)]+=votes[j]
                pr.append((ILP(vt,it2),it2))
        pr.remove(pr[0])
        pr.sort()
                    
        
        

In [13]:
print(optmar(votes,3))

(5.0, [0, 1, 2, 3])


In [14]:
print(margin(votes,3))

(5.0, [0, 1, 2, 3])
